### 크롤링

In [1]:
from bs4 import BeautifulSoup
import requests
import json
from tqdm import tqdm

In [2]:
# 단종카드 제거를 위해 카드 넘버 추출
card_category = {'CRD': 1005,
                 'CHK': 429}
headers = {"User-Agent": "Mozilla/5.0"}

# 카드 종류(key), 카드 개수(value)로 돌리기
for cat,num in tqdm(card_category.items()):
    # 신용카드인 경우, 카드 페이지 번호 추출
    if cat == 'CRD':
        url = f'https://api.card-gorilla.com:8080/v1/cards?p=1&perPage={num}&cate={cat}'
        response = requests.get(url, headers=headers)
        crd_num_json = response.json()

        crd_num_list = [int(i.get('cid')) for i in tqdm(crd_num_json.get('data')) if i.get('is_discon')==False]

        print(f'신용카드 {len(crd_num_list)}개 완료')
    
    # 체크카드인 경우, 카드 페이지 번호 추출
    elif cat == 'CHK':
        url = f'https://api.card-gorilla.com:8080/v1/cards?p=1&perPage={num}&cate={cat}'
        response = requests.get(url, headers=headers)
        chk_num_json = response.json()
        
        chk_num_list = [int(i.get('cid')) for i in tqdm(chk_num_json.get('data')) if i.get('is_discon')==False]

        print(f'체크카드 {len(chk_num_list)}개 완료')

page_list = crd_num_list+chk_num_list

 50%|█████     | 1/2 [00:39<00:39, 39.70s/it]

신용카드 769개 완료


100%|██████████| 2/2 [00:53<00:00, 26.62s/it]

체크카드 352개 완료


In [ ]:
# # 카드 정보 넣을 딕셔너리
# all_card = {}

# # 전체 카드 번호
# page_list = crd_num_list+chk_num_list

# # 카드 번호 하나씩 크롤링 시작
# for page_num in tqdm(page_list):
#     response = requests.get(f'https://api.card-gorilla.com:8080/v1/cards/{page_num}')

#     # 정상적 응답 아니면 넘어가기
#     if response.status_code !=200:
#         continue
    
#     json_soup = response.json()
    
#     # 카드 이름
#     card_name = json_soup.get('name')
#     all_card[card_name] = []

#     # 카드사 이름
#     all_card[card_name].append({'카드사' : json_soup.get('corp').get('name')})

#     # 신용인지 체크인지
#     all_card[card_name].append({'종류' :  '신용카드' if json_soup.get('cate')=='CRD' else '체크카드'})

#     # 해외 결제 여부
#     forreign_val = json_soup.get('brand')
#     all_card[card_name].append({'해외' : [ forreign_val[num].get('name') for num in range(len(forreign_val))]})

#     # 주요 혜택
#     benefit_dict = {}
#     for num in range(len(json_soup.get('key_benefit'))):
#         benefit_temp = json_soup.get('key_benefit')[num]
#         benefit_content = benefit_temp.get('info')
#         benefit_dict[benefit_temp.get('title')] = BeautifulSoup(benefit_content, 'html.parser').get_text(separator=' ', strip=True)
#     all_card[card_name].append({'혜택' : benefit_dict})

#     # 프로모션이 있는 경우 저장
#     if json_soup.get('pr_container'):
#         all_card[card_name].append({'프로모션' : BeautifulSoup(json_soup.get('pr_container'), 'html.parser').get_text(separator=' ', strip=True)})
#     else:
#         all_card[card_name].append({'프로모션' : '없음'})

#     # 연회비, 연회비 상세 추출(html이 아닌 경우 무시하고 진행됨)
#     if json_soup.get('annual_fee_basic') and json_soup.get('annual_fee_detail'):
#         all_card[card_name].append({'연회비': json_soup.get('annual_fee_basic'),
#                                                 '연회비 상세': BeautifulSoup(json_soup.get('annual_fee_detail'), 'html.parser').get_text(separator=' ', strip=True)})

In [ ]:
# with open('jinwoo_card.json', 'w', encoding='utf-8') as f:
#     json.dump(all_card, f, ensure_ascii=False, indent=4)

In [ ]:
# with open('jinwoo_card.json', 'r', encoding='utf-8') as f:
#     all_card = json.load(f)

In [ ]:
# 카드 정보 넣을 딕셔너리
all_card_list = []

# 카드 번호 하나씩 크롤링 시작
for page_num in tqdm(page_list):
    response = requests.get(f'https://api.card-gorilla.com:8080/v1/cards/{page_num}')

    # 정상적 응답 아니면 넘어가기
    if response.status_code !=200:
        continue
    
    json_soup = response.json()
    
    # 카드 이름
    card_name = f"카드 이름 : {json_soup.get('name')}"
    
    # 카드사 이름
    card_cat = f"카드사 : {json_soup.get('corp').get('name')}"

    # 신용인지 체크인지
    crd_chk = f"카드 종류 :  {'신용카드' if json_soup.get('cate')=='CRD' else '체크카드'}"

    # 해외 결제 여부
    forreign_val = json_soup.get('brand')
    card_for = f" 해외 : {[forreign_val[num].get('name') for num in range(len(forreign_val))]}"

    # 주요 혜택
    benefit_dict = {}
    for num in range(len(json_soup.get('key_benefit'))):
        benefit_temp = json_soup.get('key_benefit')[num]
        benefit_content = benefit_temp.get('comment')
        benefit_dict[benefit_temp.get('title')] = BeautifulSoup(benefit_content, 'html.parser').get_text(separator=' ', strip=True)

    card_benefit = f"혜택 : {benefit_dict}"

    # 프로모션이 있는 경우 저장
    if json_soup.get('pr_container'):
        card_promotion = f"프로모션 : {BeautifulSoup(json_soup.get('pr_container'), 'html.parser').get_text(separator=' ', strip=True)}"
    else:
        card_promotion = f"프로모션 : 없음"

    # 연회비, 연회비 상세 추출(html이 아닌 경우 무시하고 진행됨)
    if json_soup.get('annual_fee_basic') and json_soup.get('annual_fee_detail'):
        card_fee = f"'연회비': {json_soup.get('annual_fee_basic')},'연회비 상세': {BeautifulSoup(json_soup.get('annual_fee_detail'), 'html.parser').get_text(separator=' ', strip=True)}" + "</END CARD>"
    
    all_card_list.append(card_name + card_cat + crd_chk + card_for + card_benefit + card_promotion +card_fee)

        

  0%|          | 0/1121 [00:00<?, ?it/s]

C:\Users\ENVY\AppData\Local\Temp\ipykernel_15156\3206640859.py:32: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  benefit_dict[benefit_temp.get('title')] = BeautifulSoup(benefit_content, 'html.parser').get_text(separator=' ', strip=True)
 65%|██████▍   | 724/1121 [01:36<00:46,  8.45it/s]C:\Users\ENVY\AppData\Local\Temp\ipykernel_15156\3206640859.py:44: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  card_fee = f"'연회비': {json_soup.get('annual_fee_basic')},'연회비 상세': {BeautifulSoup(json_soup.get('annual_fee_detail'), 'html.parser').get_text(separator=' ', strip=True)}" + "</END CARD>"
100%|██████████| 1121/1121 [02:21<00:00,  7.89it/s]


In [31]:
all_text = ''.join(all_card_list)

### 청킹

In [ ]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# chunk_size = 1500    # 하나의 청크의 글자 수(200~1500) 일반적인 문서에서는 작게, 기술문서는 크게
# chunk_overlap = 0   # 인접 텍스트에서 중복으로 포함될 문자 수

# r_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=chunk_size,
#     chunk_overlap = chunk_overlap
# )

# texts = r_splitter.split_text(all_text)
# texts

In [40]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma # langchain에서 지원하는 chroma
from langchain_openai import OpenAIEmbeddings # Lanhchain과 연동된 openai에서 지원하는 pre-trained Embedding모델
from langchain.schema import Document
from getpass import getpass 

In [41]:
MY_API_KEY = getpass("OpenAI API Key:")


In [51]:
chunk_size = 1500    # 하나의 청크의 글자 수(200~1500) 일반적인 문서에서는 작게, 기술문서는 크게
chunk_overlap = 0   # 인접 텍스트에서 중복으로 포함될 문자 수

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap = chunk_overlap
)

texts = r_splitter.split_text(all_text)

my_chunk = [Document(page_content=text) for text in texts]

In [56]:
my_embedding = OpenAIEmbeddings(model="text-embedding-3-large",
                                api_key=MY_API_KEY
                               )

temp_embedding = my_embedding.embed_documents(texts)

In [52]:
vectordb = Chroma.from_documents(documents = my_chunk,
                                 embedding = my_embedding,
                                )

In [63]:
from langchain.chains import RetrievalQA
# Langchain 클래스들과 통합이 용이한 대화 전용 모델인 chatOpenAI 사용
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model = "gpt-3.5-turbo",
                        api_key = MY_API_KEY
                       )

my_retriever = vectordb.as_retriever(search_kwargs={"k":3})

# from_chain_type = RetrievalQA객체 성성 및 세부사항 설정
R_QA1 = RetrievalQA.from_chain_type(llm = chat_model,              # LLM모델 지정
                                #    chain_type = "refine",          # 답변을 위해 청크들이 결합되는 타입 지정
                                    # stuff(default) : 검색된 청크들의 내용을 간단하게 결합하여 답변 생성
                                    # map_reduce : 청크를 각각 처리 후 개별 결과를 결합하여 답변 생성 (문서 규모가 클 때)
                                    # refine : 이전 청크의 처리 결과를 다음 청크에 반영하여 지속적으로 응답을 개선한 후 답변 생성
                                    # map_rerank : 각 문서를 개별 평가하여 관련성을 점수화한 후, 가장 높은 문서를 선택하여 답변 생성
                                   retriever = my_retriever,       # 검색기 지정(우리는 DB까지 포함)
                                   return_source_documents = True) # 어떤 문설르 참조했는지 출력

In [64]:
question = "교통비가 부족한 30대에게 필요한 카드"
llm_response = R_QA1.invoke(question)
llm_response.get('result')

'30대에게 교통비가 부족한 경우 후불교통카드를 발급받는 것이 도움이 될 수 있습니다. 후불교통카드는 교통비를 미리 충전하지 않고 사용한 후에 나중에 한꺼번에 정산하는 형태의 카드입니다. 이를 통해 교통비를 편리하게 결제하고 관리할 수 있습니다.'

In [ ]:
from langchain_cohere import CohereRerank
from langchain.retrievers import ContextualCompressionRetriever
# JhWAU7cwOysjZmLvYN1gdp9Ym3LFko55MSJthQ4U
COHERE_API_KEY = getpass("Cohere API KEY :")

In [67]:
my_rerank = CohereRerank(cohere_api_key = COHERE_API_KEY,
                         model = "rerank-v3.5" # cohere지원 reranking모델
                        )

compression_retriever = ContextualCompressionRetriever(base_compressor = my_rerank,
                                                       base_retriever = my_retriever
                                                      )

R_QA2 = RetrievalQA.from_chain_type(llm=chat_model,
                                    chain_type = "refine",
                                    #문맥 압축검색기로 변환하여 설정
                                    retriever = compression_retriever
                                   )

In [69]:
%%time
question = "30대 해외여행을 많이 가는 여자가 쓰기 좋은 카드는?"
print(R_QA2.invoke(question).get('result'))

해외여행을 자주 다니는 30대 여성이 쓰기 좋은 카드는 "하나멤버스 Mega 체크카드"입니다. 이 카드는 VISA를 사용할 수 있어 해외에서도 편리하게 이용할 수 있습니다. 또한, 요식/마트/백화점/온라인 쇼핑/주유/병원에서 이용 시 멤버십포인트를 적립할 수 있고, SKT/KT/LGU+ 자동이체 시에도 적립이 가능합니다. 해외에서도 이용 가능하며 다양한 혜택을 받을 수 있는 이 카드가 해외여행을 자주 다니는 여성에게 적합할 것입니다.
CPU times: total: 93.8 ms
Wall time: 7.56 s
